In [1]:
import numpy as np
import torch
import os
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, Subset, ConcatDataset
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.datasets import DatasetFolder, VisionDataset
from PIL import Image
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import random
import pandas as pd
import glob
import scipy.misc
import imageio
import argparse

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
np.random.seed(10901041)
torch.manual_seed(10901041)

In [3]:
def read_masks(filepath):
    '''
    Read masks from directory and tranform to categorical
    '''
    file_list = sorted([os.path.join(filepath, x) for x in os.listdir(filepath) if x.endswith(".png")])
    n_masks = len(file_list)
    masks = np.empty((n_masks, 512, 512))

    for i, file in enumerate(file_list):
        mask = imageio.imread(file)
        mask = (mask >= 128).astype(int)
        mask = 4 * mask[:, :, 0] + 2 * mask[:, :, 1] + mask[:, :, 2]
        masks[i, mask == 3] = 0  # (Cyan: 011) Urban land
        masks[i, mask == 6] = 1  # (Yellow: 110) Agriculture land
        masks[i, mask == 5] = 2  # (Purple: 101) Rangeland
        masks[i, mask == 2] = 3  # (Green: 010) Forest land
        masks[i, mask == 1] = 4  # (Blue: 001) Water
        masks[i, mask == 7] = 5  # (White: 111) Barren land
        masks[i, mask == 0] = 6  # (Black: 000) Unknown
    # print(masks)
    return masks

def mean_iou_score(pred, labels):
    '''
    Compute mean IoU score over 6 classes
    '''
    mean_iou = 0
    for i in range(6):
        tp_fp = np.sum(pred == i)
        tp_fn = np.sum(labels == i)
        tp = np.sum((pred == i) * (labels == i))
        iou = tp / (tp_fp + tp_fn - tp)
        mean_iou += iou / 6
        # print('class #%d : %1.5f'%(i, iou))
    # print('\nmean_iou: %f' % mean_iou)

    return mean_iou

In [4]:
class Dataset(Dataset):
    def __init__(self, path, mode):
        self.images_list = sorted(
            [os.path.join(path, x) for x in os.listdir(path) if x.endswith(".jpg")]
        )
        # print(self.images_list)
        self.mode = mode
        print("reading masks...")
        # print(path)
        self.labels = read_masks(path)
        # print(self.labels)

    def __len__(self):
        return len(self.images_list)

    def __getitem__(self, i):
        sats, masks = tran(Image.open(self.images_list[i]), self.labels[i], is_train_mode=(self.mode == "train"))
        return sats, masks
def tran(sat:Image, mask: torch.Tensor, is_train_mode:bool):
    mask = Image.fromarray(mask)
    p = random.random()
    if is_train_mode:
      if p < 0.3:
        sat = transforms.functional.hflip(sat)
        mask = transforms.functional.hflip(mask)
      if p > 0.6:
        sat = transforms.functional.vflip(sat)
        mask = transforms.functional.vflip(mask)

    #[batch, 1, height, width] --> [batch, height, width]
    mask = torch.squeeze(transforms.functional.to_tensor(mask))
    mask = mask.long()
    sat = transforms.functional.to_tensor(sat)
    return sat, mask

In [5]:
# _dataset_dir = "/content/hw1_data/p3_data/"
train_set = Dataset(
    path="/kaggle/input/dlcv-hw1/hw1_data/p3_data/train", mode="train")
valid_set = Dataset(
    path="/kaggle/input/dlcv-hw1/hw1_data/p3_data/validation", mode="val")

print(len(train_set)) # Should print 2000
print(len(valid_set)) # Should print 257

/tmp/ipykernel_457/3466891986.py:10: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  mask = imageio.imread(file)


reading masks...
reading masks...
2000
257


In [6]:
train_loader = DataLoader(train_set, shuffle=True,
                          batch_size=8)
valid_loader = DataLoader(valid_set, shuffle=True,
                          batch_size=8)

In [7]:
# print(len(train_set)) # Should print 2000
# print(len(valid_set)) # Should print 257

In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
model = models.segmentation.deeplabv3_resnet50(num_classes=7)
# model.load_state_dict(torch.load('p3b_final_model.ckpt'))
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4, weight_decay=3e-5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, patience = 3, factor=0.5, mode='max', min_lr=1e-5)
cal_loss = nn.CrossEntropyLoss(label_smoothing=0.1)

cuda


In [9]:
# model.load_state_dict(torch.load("/kaggle/input/dlcv-hw1-p3b-bestmodel1013/p3b_best_model.ckpt"))
# model = model.to(device)

# model.eval()
# val_loss = []
# val_acc = []
# pred_masks_cpu = []
# labels_cpu = []
# for batch in tqdm(valid_loader):
#     images, labels = batch
#     with torch.no_grad():
#         logits = model(images.to(device))
#     logits = logits['out']
#     loss = cal_loss(logits, labels.to(device))
#     val_loss.append(loss.item())
#     pred = logits.argmax(dim=1)
#     pred_masks_cpu.append(pred.cpu())
#     labels_cpu.append(labels.cpu())
# v_loss = sum(val_loss) / len(val_loss)    
# pred_masks_numpy = torch.cat(pred_masks_cpu).numpy()
# labels_numpy = torch.cat(labels_cpu).numpy()
# valid_miou = mean_iou_score(pred_masks_numpy, labels_numpy)
# print(f"[ Val | {epoch + 1:03d}/{num_epoch:03d} ] loss = {v_loss:.5f} | valid_miou = {valid_miou:5f}")
# best_valid_miou = valid_miou

In [ ]:
# best_acc = 0
num_epoch = 100
best_valid_miou = 0
for epoch in range(num_epoch):
    # Train
    model.train()
    train_loss = []
    train_acc = []
    for batch in tqdm(train_loader):
        optimizer.zero_grad()
        images, labels = batch
        logits = model(images.to(device))
        logits = logits['out']
        labels = labels.to(device, dtype=torch.long)
        loss = cal_loss(logits, labels)
        loss.backward()
        optimizer.step()
        train_loss.append(loss.item())
    t_loss = sum(train_loss) / len(train_loss)
    # t_acc = sum(train_acc) / len(train_acc)
    print(f"[ Train | {epoch + 1:03d}/{num_epoch:03d} ] loss = {t_loss:.5f} ")

    #Val
    model.eval()
    val_loss = []
    val_acc = []
    pred_masks_cpu = []
    labels_cpu = []
    for batch in tqdm(valid_loader):
        images, labels = batch

        with torch.no_grad():
            logits = model(images.to(device))
        logits = logits['out']
        loss = cal_loss(logits, labels.to(device))
        val_loss.append(loss.item())
        pred = logits.argmax(dim=1)
#         acc = mean_iou_score(pred, labels.to(device))
#         if not torch.isnan(acc):
#             val_acc.append(acc)
        pred_masks_cpu.append(pred.cpu())
        labels_cpu.append(labels.cpu())
    v_loss = sum(val_loss) / len(val_loss)
#     v_acc = sum(val_acc) / len(val_acc)
    
    pred_masks_numpy = torch.cat(pred_masks_cpu).numpy()
    labels_numpy = torch.cat(labels_cpu).numpy()
    valid_miou = mean_iou_score(pred_masks_numpy, labels_numpy)
    
    print(f"[ Val | {epoch + 1:03d}/{num_epoch:03d} ] loss = {v_loss:.5f} | valid_miou = {valid_miou:5f}")

    scheduler.step(v_loss)
    if epoch == 0:
        torch.save(model.state_dict(), f"p3b_epoch_{epoch}.ckpt")
    if epoch == num_epoch/2:
        torch.save(model.state_dict(), f"p3b_epoch_{epoch}.ckpt")
    if epoch == num_epoch -1:
        torch.save(model.state_dict(), f"p3b_epoch_{epoch}.ckpt")
    if valid_miou > best_valid_miou:
        print(f"best model: epoch {epoch}, save model")
        torch.save(model.state_dict(), 'p3b_best_model.ckpt')
        best_valid_miou = valid_miou
#     if v_acc > best_acc:
#         print(f"best model: epoch {epoch}, save model")
# #       torch.save(model.state_dict(), f"./p3b_best_model.ckpt")
#         torch.save(model.state_dict(), 'p3b_best_model.ckpt')
#         best_acc = v_acc
    else:
        print(f"no improve: epoch {epoch}.")

  0%|          | 0/250 [00:00<?, ?it/s]

[ Train | 001/100 ] loss = 1.08538 


  0%|          | 0/33 [00:00<?, ?it/s]

[ Val | 001/100 ] loss = 0.82017 | valid_miou = 0.583430
best model: epoch 0, save model


  0%|          | 0/250 [00:00<?, ?it/s]

...

  0%|          | 0/250 [00:00<?, ?it/s]

[ Train | 076/100 ] loss = 0.52895 


  0%|          | 0/33 [00:00<?, ?it/s]

[ Val | 076/100 ] loss = 0.66544 | valid_miou = 0.739827
no improve: epoch 75.


  0%|          | 0/250 [00:00<?, ?it/s]